In [ ]:
import praw
from psaw import PushshiftAPI
import credentials as cred

username = cred.USERNAME
client_id = cred.CLIENT_ID
client_secret = cred.CLIENT_SECRET
user_agent = cred.USER_AGENT

reddit = praw.Reddit(client_id=client_id,
                     client_secret=client_secret,
                     user_agent=user_agent)
api = PushshiftAPI(reddit)

In [ ]:
from datetime import datetime
import pandas as pd

start_epoch = int(datetime().timestamp())
end_epoch = int(datetime().timestamp())

# hurricane_ida_subreddits = [
#               'hurricaneida',
#               'hurricane',
#               'tropicalweather',
#               'neworleans',
#               'louisiana',
#               'collapse',
#               'publicfreakout']
# canada_flood_subreddits = [
#             'tropicalweather',
#             'collapse',
#             'publicfreakout',
#             'canada',
#             'britishcolumbia',
#             'vancouver']
# climate_related_subreddits = [
#               'climate',
#               'climatechange',
#               'climateactionplan',
#               'ecointernet',
#               'environment',
#               'sustainability',
#               'weather']
# subreddits = hurricane_ida_subreddits + climate_related_subreddits
subreddits = []

q = "hurricane ida|hurricane|ida"
column_names = ["index", "is_post", "subreddit", "author", "post_title", "post_text", 
                "num_comments", "score", "upvote_ratio", "flair", "post_url", "post_time"]
posts_table = []
counter = 0

for subreddit in subreddits:
    results = list(api.search_submissions(q=q, after=start_epoch,before=end_epoch,
                            subreddit=subreddit,
                            filter=['subreddit', 'author', 'title', 'url'],
                            limit=1000000))
    for p in results:
        counter += 1

        # Add the post to the table
        posts_table.append([counter, "yes", p.subreddit, p.author, p.title, p.selftext, p.num_comments, p.score, p.upvote_ratio, p.link_flair_text, p.permalink,
                            datetime.utcfromtimestamp(p.created)])

        # Add the comments (only top level ones)
        for comment in p.comments:
            # retrieve all comments using more API calls if needed
            p.comments.replace_more(limit=1000)
            posts_table.append([counter, "no", p.subreddit, comment.author, p.title, comment.body, "", comment.score, "", "", comment.permalink,
                                datetime.utcfromtimestamp(comment.created)])

reddit_df = pd.DataFrame(posts_table, columns=column_names)
reddit_df_cp = pd.DataFrame(posts_table, columns=column_names)

In [ ]:
reddit_df.shape

In [ ]:
reddit_df.head(25)

In [ ]:
reddit_df["post_text"] = reddit_df["post_text"].str.replace('\n', '')
reddit_df["post_title"] = reddit_df["post_title"].fillna("")
reddit_df["post_text"] = reddit_df["post_text"].fillna("")
reddit_df["flair"] = reddit_df["flair"].fillna("")
reddit_df["author"] = reddit_df["author"].fillna("")

In [ ]:
reddit_df["post_time_year"] = reddit_df["post_time"].apply(lambda x: x.year)
reddit_df["post_time_month"] = reddit_df["post_time"].apply(lambda x: x.month)
reddit_df["post_time_day"] = reddit_df["post_time"].apply(lambda x: x.day)

In [ ]:
labels = reddit_df.loc[(reddit_df["post_text"] == "[deleted]") |
                       (reddit_df["post_text"] == "[removed]")].index
reddit_df.drop(reddit_df.index[labels], inplace=True)
reddit_df.drop_duplicates(inplace=True)

In [ ]:
reddit_df.isna().sum()

In [ ]:
reddit_df.shape

In [ ]:
reddit_df.head()

In [ ]:

reddit_df.to_csv("", index=False)